In [1]:
#Package Dependencies
import pandas as pd
import datetime as dt
from IPython.display import display
import csv
from collections import namedtuple
from pprint import pprint

In [2]:
#SFTP for data access
import paramiko
host = "db3.pharmetrics.com"                    
port = 22
password = "OXgate1409"                  
username = "camendol"   
transport = paramiko.Transport((host, port))
transport.connect(username = username, password = password)
db3_sftp = paramiko.SFTPClient.from_transport(transport)

In [3]:
# database connection
from sqlalchemy import create_engine 
sql_engine = create_engine('sqlite:///C:\\Users\\camendol\\Documents\\PI360\\pi360_aae001.db')

In [4]:
#Parse the schema json for the file build
import json
schema_lib="C:\\Users\\camendol\\Documents\\PI360\\Python_Avro\\claims_layout.avsc"
json_data=open(schema_lib).read()
schema_data = json.loads(json_data)
#Pare down the json to a simple dict for csv-file variable management
#Create all csv handling logic from the avro schema
attrib_dict={}
for schema_attribs in schema_data['fields']:
    #Dictionary var->type
    attrib_dict[schema_attribs['name']]=schema_attribs['type'][0]
#Build the vars list from the schema    
vars=list(attrib_dict.keys())

In [5]:
src_claim_raw='/incoming/pi360/aae/001/Epic_billing_in.CSV'
src_enroll_raw='/incoming/pi360/aae/001/Epic_patient_out.csV'

In [6]:
data_delm='|'
def page (reader,pagesize):
    """ 
    Page generator. Take a CSV `reader` and yield
    `page` sized blocks of data. 
    """
    page = []
    for i, line in enumerate(reader):
        if (i % pagesize == 0 and i > 0):
            yield page
            del page[:]
    yield page

In [ ]:
#Claims Data Acquisition
start = dt.datetime.now()
j = 0
index_start = 1
chunksize=20000

infile=db3_sftp.open(src_claim_raw,'rt') 
#Get Header Data
header=infile.readline().split(data_delm)
print(header)
obs_block=page(infile,chunksize)
for rowset in obs_block:
    #if j>1: break
    temp_df=pd.DataFrame(rowset,columns=header)
    temp_df['admit_date'] = pd.to_datetime(temp_df['admit_date'] ) 
    temp_df['discharge_date'] = pd.to_datetime(temp_df['discharge_date'])

    temp_df.index += index_start   
    
    j+=1
    print ('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

    temp_df.to_sql('claims_data', sql_engine, if_exists='append')
    index_start = temp_df.index[-1] + 1   

In [ ]:
temp_df.head()

In [ ]:
temp_df.columns

In [7]:
data_delm='|'
start = dt.datetime.now()
j = 0
index_start = 1
chunksize=10000

infile=db3_sftp.open(src_enroll_raw,'rt') 
#Get Header Data
header=infile.readline().split(data_delm)
print(header)
obs_block=page(infile,chunksize)
for rowset in obs_block:
    #if j>1: break
    temp_df=pd.DataFrame(rowset,columns=header)

    temp_df.index += index_start   
    
    j+=1
    print ('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

    temp_df.to_sql('enroll_data', sql_engine, if_exists='append')
    index_start = temp_df.index[-1] + 1   

['patient_num                                                                                                                                                                                                                                                     ', 'I', 'p', '    ', 'NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2nQfJhUF75zNdMc1dkHAZXXd09xrROQYDIMbPrqqOTqBjyi7yvwA55RpalZtXjRgrh74lUIySF6+trHBxJkRLZpq3SAMIO+MrXEHmpfjptM0JyrQeXZqhQvVsYfDSKfmOSZVOmbvo4K3097TK5GAQjkX8Szx0TXf6IdcK34ne86P0F7vQsIcUZQye42cLkhwms1FTNRhMDD10Wqddaw1CwBpfCDdj3T+2tCGxLf5uc58eil2/tA34MBUbg+Hg6fZmbVxhPnwIfSpiZL/Sl0xvJbjm9ug4pNJ1zMBMl8BA==', 'pat', '                                                                                                            ', '001', '                                                                                                            ', '001', '8s1Bg80Zvw60yBGml3PNJ18sC20GwHRwbe+G6R2rTCDfYZvkZuEvE07sPY/H7SQy0lqBq+Z+eC5MXOe7GxDXplE4FotsKl4s4XwHNybrHL4=', '000', '-002', '

In [9]:
query_code= 'select * from enroll_data limit 10'
df = pd.read_sql_query(query_code, sql_engine)
df

,index,patient_num,I,p,,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2nQfJhUF75zNdMc1dkHAZXXd09xrROQYDIMbPrqqOTqBjyi7yvwA55RpalZtXjRgrh74lUIySF6+trHBxJkRLZpq3SAMIO+MrXEHmpfjptM0JyrQeXZqhQvVsYfDSKfmOSZVOmbvo4K3097TK5GAQjkX8Szx0TXf6IdcK34ne86P0F7vQsIcUZQye42cLkhwms1FTNRhMDD10Wqddaw1CwBpfCDdj3T+2tCGxLf5uc58eil2/tA34MBUbg+Hg6fZmbVxhPnwIfSpiZL/Sl0xvJbjm9ug4pNJ1zMBMl8BA==,pat,,001,8s1Bg80Zvw60yBGml3PNJ18sC20GwHRwbe+G6R2rTCDfYZvkZuEvE07sPY/H7SQy0lqBq+Z+eC5MXOe7GxDXplE4FotsKl4s4XwHNybrHL4=,...,11101,11001,11000,02910,facility_name,facility_npi,patient_health_plan_id_pri,patient_health_plan_id_sec,patient_num,
0,1,0008547 ...,2,2,1959,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,331,KMPlrsYEdjPZEGgplZxvuEfHf1rMPQrygCndKhf2g0+MHg...,000,zPx2Tz8LhmHQ3L8/Ihha5PIayurq+zsQSRWiCDs0Vkr1WR...,...,00000,00000,00000,02910,ABLEH PARENT ...,...,267558882A ...,3363254024 ...,0008547 ...,\n
1,2,8004626 ...,2,2,1967,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,330,kDiVCUtMy5NvM93dQYz2edMZm09AvgSiSGKvbwTZkr7l+O...,000,UCb9lFdnwYD/HuCBdqXoOjXJ6J+Wunxi1P3DvfMydSpTp4...,...,00000,00000,00000,02910,UMHC PARENT ...,...,W181072488 ...,...,8004626 ...,\n
2,3,20590923 ...,2,2,2003,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,330,yKEtLC7GGJ4GSM+7xamuT6Nw8vLbXjVkcrtoB0vDdkycw2...,000,7Hj1tbQI6sfeV+lXjwXK1k/stPN/z3flUbFA3SIuBNjeE6...,...,00000,00000,00000,02910,ABLEH PARENT ...,...,592-71-8168 ...,...,20590923 ...,\n
3,4,8001232 ...,2,2,1965,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,331,hY1pE1XoUWVcsNkaCiFq5gqGPwJ+h2xqK2KAP1P+XZ7D1D...,000,L9xhfhd4F2+gwdHsJCZsTxp+iEEQCe+RlZl1bQvN0ytGd4...,...,00000,00000,00000,02910,ABLEH PARENT ...,...,A5210857000 ...,...,8001232 ...,\n
4,5,8001222 ...,2,2,1942,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,334,tCX4llCmn0WK2ufMoX0e8OeW4uHJR81z3pivA+xuIbKZrT...,000,HoI6+vbpVLSbm7wssp8Xtpq7eTFDVOgK3Ok+kHma433jrJ...,...,00000,00000,00000,02910,UMHC PARENT ...,...,H07904057 ...,...,8001222 ...,\n
5,6,20015284 ...,1,1,1992,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,331,90QHhqLgrLMA1/hC/yJza6qsZUJEKmjYZk+jKkf4vslQpi...,000,rurWePRzz47pJPKLiG47X7l3Mp04VRuwmeSjMrgFSqnqu9...,...,00000,00000,00000,02910,UMHC PARENT ...,...,790317877 ...,...,20015284 ...,\n
6,7,20666234 ...,2,2,1994,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,330,55+9V/f8MtR0sckXn+oIXA6X75SGLzjUPvRE6vOm4I+Up/...,000,j/jLTw1Imb7T1CJcViKkoysRkX2HjOA6HOxVW2AAr4YYql...,...,00000,00000,00000,02910,UMHC PARENT ...,...,...,...,20666234 ...,\n
7,8,20558943 ...,1,1,2003,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,330,vc/m33gB7dbm1kz6nXKIhdVCH5MYZnw9g/7XQE7R92jMfY...,000,LQt9VD/6yYS8yudFQBC/3E+cmjFl8P8vFC3TLX0cHJU6ae...,...,00000,00000,00000,02910,ABLEH PARENT ...,...,7905958566 ...,...,20558943 ...,\n
8,9,8001294 ...,2,2,1959,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,331,R2qOQMj64tHqJ729YucmlWLXmOwqhUJXzT2mbpHoTypcmk...,000,Zzm6TAuxVRgB45EN2Qy4V2jP8Pw3jwlQJ0oQjMY8ugEIR0...,...,00000,00000,00000,02910,ABLEH PARENT ...,...,847123991 ...,...,8001294 ...,\n
9,10,8660263 ...,2,2,1964,NinaIxwj5kh87MnJVQCmSUwGBWjjfnofW06As3j2iwZMn2...,331,MK6ySDvNJkT+I1xIWmKc3AIOz1J3d1ts6xP5SGZTD8H52Z...,000,MK8ZhrXEi6Mheh1Vm/IlFsLsT3Luwqy51yqOHYxrgPn26m...,...,00000,00000,00000,02910,ABLEH PARENT ...,...,7907487382 ...,...,8660263 ...,\n


In [ ]:
query_code= 'select * from claims_data limit 10'
df = pd.read_sql_query(query_code, sql_engine)
df

In [ ]:
sql_engine.execute('drop table enroll_data')